## En este notebook vamos a analizar el comportamiento de los usuarios y sus actividades dentro de la pagina.

In [1]:
#Inlcuimos las librerias que vamos a utilizar
import numpy as np 
import pandas as pd
import datetime as datetime

# plots
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
%matplotlib inline

In [2]:
#Cargamos los datos
events = pd.read_csv('events.csv', low_memory = False)

In [3]:
#categorizamos
events['timestamp'] = pd.to_datetime(events['timestamp'], errors = 'coerce', format= '%Y-%m-%d')
events['event'] = events['event'].astype('category')

In [5]:
events['person'].isnull().value_counts()

False    1011288
Name: person, dtype: int64

In [12]:
events.groupby('person')['event'].value_counts().sort_values(ascending = False).head(10)

person    event         
2d5a84c1  viewed product    1525
3e34b3cf  viewed product    1487
3b5a5833  viewed product    1383
71492f2b  viewed product    1364
d7e60792  viewed product    1212
cd6e0b8d  viewed product    1174
71492f2b  brand listing     1149
6b07be7b  viewed product    1118
285101a1  viewed product    1077
c501ff3a  viewed product    1003
Name: event, dtype: int64